In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatbot-ai-q-and-a/AI.parquet
/kaggle/input/chatbot-ai-q-and-a/AI.csv
/kaggle/input/python-dataset/Python codes.csv
/kaggle/input/gpt2/keras/gpt2_medium_en/2/config.json
/kaggle/input/gpt2/keras/gpt2_medium_en/2/tokenizer.json
/kaggle/input/gpt2/keras/gpt2_medium_en/2/metadata.json
/kaggle/input/gpt2/keras/gpt2_medium_en/2/model.weights.h5
/kaggle/input/gpt2/keras/gpt2_medium_en/2/assets/tokenizer/merges.txt
/kaggle/input/gpt2/keras/gpt2_medium_en/2/assets/tokenizer/vocabulary.json


In [2]:
!pip install transformers datasets evaluate accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 20.4 MB/s eta 0:00:00


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
print(device)

cpu


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json
import csv

class pythonDataset(Dataset):
    def __init__(self, jokes_dataset_path = '/kaggle/input/chatbot-ai-q-and-a'):
        super().__init__()

        short_jokes_path = os.path.join(jokes_dataset_path, 'AI.csv')

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"
        
        with open(short_jokes_path) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
              
            next(csv_reader, None) 
            
            for row in csv_reader:
                # Check if row has the expected number of columns
                if len(row) > 1:
                    joke_str = f"data:{row[1]}{self.end_of_text_token}"
                    self.joke_list.append(joke_str)
        
    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]

In [7]:
dataset = pythonDataset()
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [8]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-6
WARMUP_STEPS = 500
MAX_SEQ_LEN = 40

In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup

model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = len(data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=num_training_steps)

proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_paragraphs_tens = None
models_folder = "/kaggle/working/trained_models"  # folder for the trained model

if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx, paragraph in enumerate(data_loader):
        
        #################### Fit as many paragraph sequences into MAX_SEQ_LEN sequence as possible ####
        paragraph_tens = torch.tensor(tokenizer.encode(paragraph[0])).unsqueeze(0).to(device)
        
        if paragraph_tens.size()[1] > MAX_SEQ_LEN:
            paragraph_tens = paragraph_tens[:, :MAX_SEQ_LEN]
        
        if not torch.is_tensor(tmp_paragraphs_tens):
            tmp_paragraphs_tens = paragraph_tens
            continue
        else:
            if tmp_paragraphs_tens.size()[1] + paragraph_tens.size()[1] > MAX_SEQ_LEN:
                work_paragraphs_tens = tmp_paragraphs_tens
                tmp_paragraphs_tens = paragraph_tens
            else:
                tmp_paragraphs_tens = torch.cat([tmp_paragraphs_tens, paragraph_tens[:, 1:]], dim=1)
                continue
                
        outputs = model(work_paragraphs_tens, labels=work_paragraphs_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss += loss.detach().data
                       
        proc_seq_count += 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss is {sum_loss}")
            batch_count = 0
            sum_loss = 0.0
    
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_pythonlecturer_{epoch}.pt"))

EPOCH 0 started==============================
EPOCH 1 started==============================
EPOCH 2 started==============================
EPOCH 3 started==============================
EPOCH 4 started==============================


In [12]:
def choose_from_top(probs, n=2, random_seed=None):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    np.random.seed(random_seed)
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [13]:
MODEL_EPOCH = 4
model_path = os.path.join(models_folder, f"gpt2_medium_pythonlecturer_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

generated_paragraph = 5
first_sentence = "Q:Who did the first work generally recognized as AI? \n\n A:"
max_paragraph_length = 100

paragraphs_output_file_path = os.path.join("/kaggle/working/", f'generated_lectures_{MODEL_EPOCH}.txt')
audio_file_folder = "/kaggle/working/"
if os.path.exists(paragraphs_output_file_path):
    os.remove(paragraphs_output_file_path)

randomness = None
model.eval()
with torch.no_grad():
    for paragraph_idx in range(generated_paragraph):
        paragraph_finished = False
        cur_ids = torch.tensor(tokenizer.encode(first_sentence)).unsqueeze(0).to(device)
        for i in range(max_paragraph_length):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0, -1], dim=0)
            if i < 3:
                n = 20
            else:
                n = 3
                
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n, random_seed=randomness)

            cur_ids = torch.cat([cur_ids, torch.ones((1, 1)).long().to(device) * next_token_id], dim=1)

            if next_token_id in tokenizer.encode('.'):
                paragraph_finished = True
                break

        if paragraph_finished:
            output_list = list(cur_ids.squeeze().to('cpu').numpy())
            output_text = tokenizer.decode(output_list)
            print(output_text + "\n")

            with open(paragraphs_output_file_path, 'a') as f:
                f.write(f"{output_text} \n\n")

2024-08-08 19:09:20.464709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 19:09:20.464817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 19:09:20.586230: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Q:Who did the first work generally recognized as AI? 

 A:I don't know, but it's a very important question.

Q:Who did the first work generally recognized as AI? 

 A:I don't think anyone really has any clue who did it, but the earliest known mention of it is in 1878 by the German physicist, Carl von Löw.

Q:Who did the first work generally recognized as AI? 

 A:I think we have to look back at the history of the field.

Q:Who did the first work generally recognized as AI? 

 A:The first work that was recognized as AI was the work of the German mathematician and philosopher Gottfried Wilhelm Leibniz, who published the first formal definition of AI in his book The Theory of Machines.

Q:Who did the first work generally recognized as AI? 

 A:Babbage.

